<a href="https://colab.research.google.com/github/d22cs051/dlops-23/blob/main/assignment/D22CS051_Lab_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1

## Question 1

### getting iris dataset with sklearn

In [206]:
from sklearn.datasets import load_iris
data = load_iris()
X,y = data.data, data.target
class_names = data.target_names
X.shape,y.shape,class_names.shape

((150, 4), (150,), (3,))

### importing *modules*

In [207]:
import numpy as np

### making single perceptron

In [208]:
# init weights
w = np.random.uniform(low=-0.3,high=0.3,size=(4,))
b = 1
w,b

(array([ 0.08550636, -0.02367256, -0.22582373, -0.08744008]), 1)

### training loop

In [209]:
EPOCHS = 3
lr = 0.01
for epoch in range(EPOCHS):
  loss = 0
  for x,target in zip(X,y):
    # forward pass
    # print(x.shape,w.shape)
    forward =  np.matmul(w.T,x) + b
    # print(f"pred: {forward}, actual: {target}")
    
    # calculate the loss (L2 Loss)
    loss += (target - forward)**2

  print("loss", loss)
  # backpropagation
  b_o_i = np.array((2*(loss)*-1)/len(X))
  # print("b_o_i shape",b_o_i.shape)
  print(f"old w {w}, old b {b}")
  # updating the weights and baises
  w += lr*b_o_i
  b += lr*b_o_i
  print(f"new w {w}, new b {b}")    
  print()

loss 258.5348526781396
old w [ 0.08550636 -0.02367256 -0.22582373 -0.08744008], old b 1
new w [ 0.05103505 -0.05814387 -0.26029504 -0.12191139], new b 0.9655286863095813

loss 416.8476345172534
old w [ 0.05103505 -0.05814387 -0.26029504 -0.12191139], old b 0.9655286863095813
new w [-0.00454463 -0.11372355 -0.31587473 -0.17749108], new b 0.9099490017072809

loss 845.1154144508333
old w [-0.00454463 -0.11372355 -0.31587473 -0.17749108], old b 0.9099490017072809
new w [-0.11722669 -0.22640561 -0.42855678 -0.29017313], new b 0.7972669464471698



## Question 2

### making multilayer perceptron

In [210]:
# making y -> (0,1,2..) -> [[1,0,0],[0,1,0],[0,0,1]...]
# print(y)
y_new = []
for i in y:
  if i == 0:
    y_new.append([1,0,0])
  elif i == 1:
    y_new.append([0,1,0])
  elif i == 2:
    y_new.append([0,0,1])
# print(y_new)
y = np.array(y_new)

In [211]:
# ARCH. Of MLP layer
# 4 input features -> 4 (hidden layer 1/activation = relu) -> 5 (hidden layer 2/activation = softmax) -> 3 (output layer)

# inti weights
# convention w_x_y - > weights b/w x & y layer
w_i_h1 = np.random.uniform(low=-0.3,high=0.3,size=(4,4))
w_h1_h2 = np.random.uniform(low=-0.3,high=0.3,size=(5,4))
w_h2_o = np.random.uniform(low=-0.3,high=0.3,size=(3,5))

# bias inti
bl1 = np.zeros((4,1))
bl2 = np.zeros((5,1))
bl3 = np.zeros((3,1))

In [212]:
def ReLU(X:np.ndarray):
  return np.maximum(0,X)

def d_ReLU(X:np.ndarray):
  return np.array([(0 if i<0 else 1) for i in X])

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [213]:
def loss_fn(y_pred:np.ndarray,y:np.ndarray):
  return np.sum(y-y_pred)**2

In [215]:
from tqdm.auto import tqdm

# training loop
EPOCHS = 15
lr = 0.01
for epoch in tqdm(range(EPOCHS)):
  loss = 0
  for x,target in zip(X[:10],y[:10]):
    x.shape += (1,)
    target.shape += (1,)
    # forward pass
    l1 = np.matmul(w_i_h1,x) + bl1
    # print(l1)
    # print(l1.shape)
    l1_o = ReLU(l1)
    # print(l1_o,l1_o.shape)
    l2 = np.matmul(w_h1_h2,l1_o) +bl2
    # print(l2)
    # print(l2.shape)
    l2_o = ReLU(l2)
    # print(l2_o,l2_o.shape)
    l3_o = np.matmul(w_h2_o,l2_o) + bl3 # preds logits

    # calculte the loss
    loss += loss_fn(l3_o,target)
  
  # backprop.
  print(f"loss: {loss}")
  delta_o = l3_o - target
  # print(delta_o.shape,l2_o.T.shape)
  w_h2_o += -lr * delta_o @ l2_o.T
  bl3 += -lr * delta_o

  delta_h2 = w_h2_o.T @ delta_o * d_ReLU(l1_o)
  # print(delta_h2.shape,l1_o.shape)
  w_h1_h2 += -lr * delta_h2 
  bl2 += -lr * delta_h2 @ l1_o

  # print(w_i_h1.shape,delta_h2.shape)
  delta_h1 = delta_h2 @ w_i_h1 * d_ReLU(x)
  # print(delta_h1.shape,l1_o.shape)
  w_h1_h2 += -lr * delta_h2 
  bl2 += -lr * delta_h2 @ l1_o

  0%|          | 0/15 [00:00<?, ?it/s]

loss: 7.471102299162561
loss: 7.3224273634092265
loss: 7.1767110588773795
loss: 7.033894508805723
loss: 6.893920008080486
loss: 6.756730999919689
loss: 6.622272053021287
loss: 6.49048883916616
loss: 6.3613281112667535
loss: 6.234737681852548
loss: 6.110666401983681
loss: 5.9890641405842056
loss: 5.869881764186581
loss: 5.7530711170792665
loss: 5.638585001849388
